# Download AEF Embeddings

In [1]:
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

from mangroves.embeddings import Embeddings
from mangroves.collection import Collection

In [3]:
# project = os.getenv('GEE_PROJECT_KEY')
with open('/home/nus/workspace/tutorial/GEE/gee_project_key.txt', "r") as f:
    project = f.readline().strip()

collection = Collection(project=project)

2025-11-13 16:13:39,949 - INFO - Google Earth Engine initialized successfully with service account


In [4]:
input_samples = pd.read_csv('../share/data/samples_mangrove.csv')
latitude_deg = input_samples['latitude'].tolist()[0]
longitude_deg = input_samples['longitude'].tolist()[0]
year = input_samples['year'].tolist()[0]

file_configs = [
    {
        'description': 'test',
        'file': os.path.join(os.getcwd(), '../share/data/samples_mangrove.csv'),
        'strategy': 'later',
        'output_dir': os.path.join('/home/nus/Downloads/gee_outputs/'),
        'prefix': 'test_'
    },
]

latitude_deg, longitude_deg, year

(3.8046593211, 98.6429308016, 2020)

In [5]:
embeddings = Embeddings()
embeddings.from_collection(
    latitude_deg, 
    longitude_deg, 
    year, 
    REGION_DIAMETER_P, 
    SPATIAL_RESOLUTION_M, 
    collection
)

2025-11-13 16:13:40,349 - INFO - Filtered images for 2020: 1
2025-11-13 16:13:40,824 - INFO - Successfully extracted 64 bands
2025-11-13 16:13:40,826 - INFO - Successfully created patch with shape: (64, 1, 1)


In [6]:
# plt.imshow(embeddings.data[0:3].transpose(1, 2, 0), cmap='viridis')

In [ ]:
from tqdm import tqdm

filename = '../share/data/samples_mangrove.csv'
output_path = '/home/nus/Downloads/embeddings/'
os.makedirs(output_path, exist_ok=True)

try:
    df_input = pd.read_csv(filename)
except Exception as e:
    print(f' Error reading file {filename}: {e}')

df_output = []
for idx, row in tqdm(df_input.iterrows(), total=df_input.shape[0]):
    latitude = row.get('latitude')
    longitude = row.get('longitude')
    year = row.get('year')
    
    embeddings = Embeddings()
    embeddings.from_collection(
        latitude_deg, 
        longitude_deg, 
        year, 
        REGION_DIAMETER_P, 
        SPATIAL_RESOLUTION_M, 
        collection
    )

    df_output.append({
        'latitude': latitude,
        'longitude': longitude,
        'year': year
    })
    band_names = collection.band_names
    for i, band_name in  enumerate(band_names):
        df_output[-1][band_name] = embeddings.data[i].flatten()[0]

    # Rate limiting between features to avoid quota issues
    time.sleep(1)

df_output = pd.DataFrame(df_output)
output_file = os.path.join(output_path, 'embeddings_mangrove.csv')
df_output.to_csv(output_file, index=False)
print(f' Embeddings saved to {output_file}')

  0%|          | 0/1 [00:00<?, ?it/s]2025-11-13 16:14:52,244 - INFO - Filtered images for 2020: 1
2025-11-13 16:14:52,647 - INFO - Successfully extracted 64 bands
2025-11-13 16:14:52,648 - INFO - Successfully created patch with shape: (64, 1, 1)
100%|██████████| 1/1 [00:02<00:00,  2.88s/it]

 Embeddings saved to /home/nus/Downloads/embeddings/embeddings_mangrove.csv
